In [6]:
import numpy as np
import pandas as pd
import csv


In [7]:
# mount my google drive
from google.colab import drive
from os import listdir, mkdir
from os.path import isfile, join, exists
drive.mount('/content/gdrive/')
folder = '/content/gdrive/My Drive/FinTechFinal/Data'
print(listdir(folder))


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
['stock_info_1.csv', 'txn_2019H1.zip', 'txn_2019H2.zip', 'txn_2020H1.zip', 'txn_2020H2.zip', 'stock_info.zip', 'stock_category.csv', 'txn_2021Q1.csv', 'stock_info_2021Q1.csv', 'stock_category.xlsx', 'cust_info_1.csv', '檔案說明_20200423.xlsx', 'txn_2019H1', 'models', 'cust_txn_result_final.csv', 'test.csv', 'train.csv', 'result.csv']


In [8]:
# Read Data
custom_info = pd.read_csv(join(folder, "cust_info_1.csv"))
custom_info.head()

txn = pd.read_csv(join(folder, "cust_txn_result_final.csv"))
custom_info = pd.merge(custom_info,txn,on='cust',how='left')
custom_info.head()

,cust,age_level,open_acct_year,source_code,breach_ind,breach_date,breach_rank,investment_exp_code,buy_cnt,sell_cnt,nontxn_cnt,txn_amount,cust_size
0,0xDB5D3FF6B7FE584CAE62A6C482194282E627EE8F9BBB...,3,2018,B,1,560.0,1.0,2.0,26.0,24.0,5.0,617670.0,1.0
1,0x1879388A362A9CE3DB27F651C333F9BF4B66460538CF...,4,2016,B,1,556.0,1.0,3.0,95.0,81.0,1.0,255890.0,1.0
2,0x78D069BCC23EAD3DF21154E5C04838C7F942ACD422BB...,3,2020,A,1,555.0,1.0,0.0,7.0,1.0,0.0,90320.2,1.0
3,0x2E5D5882D3FA49E0B552A4C481F43307F854246B918B...,3,2020,B,1,552.0,1.0,0.0,19.0,14.0,2.0,1982480.0,1.0
4,0x6711C00180B590285AB040EAAB3EB9D1F7704BA2B4CE...,2,2020,B,1,546.0,1.0,0.0,6.0,3.0,3.0,31783.2,1.0


In [9]:

#normalize
max_year = custom_info["open_acct_year"].max()
min_year = custom_info["open_acct_year"].min()
custom_info["open_acct_year"] = (custom_info["open_acct_year"] - min_year)/(max_year - min_year)
max_buy = custom_info["buy_cnt"].max()
min_buy = custom_info["buy_cnt"].min()
custom_info["buy_cnt"] = (custom_info["buy_cnt"] - min_buy)/(max_buy - min_buy)
max_sell = custom_info["sell_cnt"].max()
min_sell = custom_info["sell_cnt"].min()
custom_info["sell_cnt"] = (custom_info["sell_cnt"] - min_sell)/(max_sell - min_sell)
# Set input, output
data_selected = custom_info[["age_level", "open_acct_year", "buy_cnt", "sell_cnt", "breach_ind", "cust_size"]] #全部使用的欄位
data_selected = data_selected.dropna() #drop missing data

X = data_selected[["age_level", "open_acct_year", "buy_cnt", "sell_cnt", "cust_size"]] # input的欄位
y = data_selected["breach_ind"]


# Split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train = pd.DataFrame(y_train).reset_index(drop=True)
y_positvie  = y_train[y_train['breach_ind'] == 1]
X_train = X_train.reset_index(drop=True)
X_positvie = X_train.iloc[list(y_positvie.index)]
X_positvie = pd.concat([X_positvie]*(int(len(X_train) /len(X_positvie))), ignore_index = True)
X_train = pd.concat([X_positvie, X_train], ignore_index=True)
y_positvie = pd.concat([y_positvie]*(int(len(y_train) /len(y_positvie))), ignore_index = True)
y_train = pd.concat([y_positvie, y_train], ignore_index=True)
y_train = y_train['breach_ind']

In [10]:
train_data = X_train.copy()
train_data['breach_ind'] = pd.DataFrame(y_train)
train_data.to_csv(join(folder, 'train.csv'))
test_data = X_test.copy()
test_data['breach_ind'] = pd.DataFrame(y_test)
test_data.to_csv(join(folder, 'test.csv'))